In [7]:
# +++ REFERENCE DO NOT DELETE +++
# +++ Calculation CHECKED against: http://www.tangotools.com/ui/UlcerIndex.xls
# +++ see C:\Users\ping\Google Drive\stocks\ulcer_index_check\UlcerIndex.xlsx

In [8]:
import pandas as pd
import numpy as np
from ulcer_index_vectorized import ulcer_index_vectorized
from ulcer_index import ulcer_index
from util import symb_perf_stats
np.set_printoptions(precision=6)

In [9]:
file_csv = 'C:/Users/ping/Google Drive/stocks/ulcer_index_check/df_close.csv'
df_close = pd.read_csv(file_csv, index_col=0)
print(df_close.head(3), '\n')
print(df_close.tail(3), '\n')
print(df_close.columns, '\n')
print('len(df_close): {}\n'.format(len(df_close)))

                 GE     AAPL     MSFT       PG      XOM
date                                                   
2019-05-08  10.1200  199.738  123.871  102.436  71.7024
2019-05-09  10.0005  197.592  123.861  101.626  71.6371
2019-05-10  10.0902  194.855  125.470  103.352  72.2601 

              GE     AAPL    MSFT      PG      XOM
date                                              
2020-05-06  5.98  299.818  182.54  113.10  43.1533
2020-05-07  6.11  302.920  183.60  112.17  43.3985
2020-05-08  6.29  310.130  184.68  115.95  45.3016 

Index(['GE', 'AAPL', 'MSFT', 'PG', 'XOM'], dtype='object') 

len(df_close): 254



In [16]:
%%timeit
symbols, period_yr, drawdown, UI, max_drawdown, \
df_symbols_returns_std, Std_UI, CAGR, CAGR_Std, CAGR_UI = ulcer_index_vectorized(df_close)
print('drawdown:\n{}\n{}\n'.format(drawdown, type(drawdown)))
print('len(drawdown):\n{}\n{}\n'.format(len(drawdown), type(len(drawdown))))
print('max_drawdown:\n{}\n{}\n'.format(max_drawdown, type(max_drawdown)))
print('UI:\n{}\n{}\n'.format(UI, type(UI)))

 -0.081146 -0.029736  0.000000 -0.011103
2019-05-14  0.000000 -0.066602 -0.018881 -0.004804 -0.009797
...              ...       ...       ...       ...       ...
2020-05-04 -0.527645 -0.104034 -0.049664 -0.083358 -0.399585
2020-05-05 -0.528406 -0.090587 -0.039461 -0.081458 -0.400254
2020-05-06 -0.545140 -0.081206 -0.030002 -0.104499 -0.411492
2020-05-07 -0.535252 -0.071700 -0.024370 -0.111862 -0.408148
2020-05-08 -0.521560 -0.049605 -0.018631 -0.081933 -0.382194

[254 rows x 5 columns]
<class 'pandas.core.frame.DataFrame'>

len(drawdown):
254
<class 'int'>

max_drawdown:
GE     -0.545140
AAPL   -0.314274
MSFT   -0.280393
PG     -0.231557
XOM    -0.579255
dtype: float64
<class 'pandas.core.series.Series'>

UI:
GE      0.220714
AAPL    0.088590
MSFT    0.071044
PG      0.051234
XOM     0.217040
dtype: float64
<class 'pandas.core.series.Series'>

drawdown:
                  GE      AAPL      MSFT        PG       XOM
date                                                        
2019-05-08 

In [11]:
%%timeit 
for symbol in df_close.columns:
    close = df_close[symbol].values
    drawdown, UI, max_drawdown = ulcer_index(close)  # NaN is replaced with zero
    # print(symbol, type(close))
    # print(close, '\n')
    # print('len(drawdown): {}'.format(len(drawdown)))
    # print('max_drawdown): {:<10.6f}'.format(max_drawdown))
    # print(drawdown[:3], '...', drawdown[-3:], '\n', 'UI: {:<10.6f}\n'.format(UI))

282 µs ± 15.6 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [12]:
%%timeit 
for symbol in df_close.columns:
    close = df_close[symbol].values
    period_yr, CAGR, CAGR_Std, CAGR_UI, daily_return_std, Std_UI, \
    drawdown, ulcer_index, max_drawdown = \
    symb_perf_stats(close)

701 µs ± 78 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [13]:
# check UI: 0.220714, max_drawdown: -0.545140, len(drawdown): 254
close = df_close['GE'].values
drawdown, UI, max_drawdown = ulcer_index(close)  # NaN is replaced with zero
print('close: ', close[:3], '...', close[-3:])
print('len(drawdown): {}'.format(len(drawdown)))
print('max_drawdown: {:<10.6f}'.format(max_drawdown))
print('drawdown: ', drawdown[:3], '...', drawdown[-3:])
print('UI: {:<10.6f}\n'.format(UI))

close:  [10.12   10.0005 10.0902] ... [5.98 6.11 6.29]
len(drawdown): 254
max_drawdown: -0.545140 
drawdown:  [-0.       -0.011808 -0.002945] ... [-0.54514  -0.535252 -0.52156 ]
UI: 0.220714  



In [14]:
def myUI(close):
    first_close = close[0]
    close_count = len(close)
    cum_return = close / first_close
    # calculate percent drawdown
    drawdown = -1 * (1 - cum_return / np.maximum.accumulate(cum_return))
    max_drawdown = drawdown.min()
    drawdown_square = np.square(drawdown)
    sum_drawdown_square = np.sum(drawdown_square)
    UI = np.sqrt(sum_drawdown_square / close_count)

    daily_return = close[1:] / close[:-1] - 1
    daily_return_std = np.std(daily_return, ddof=1)
    # period_yr = len(close) / 252  # 252 trading days in a year
    period_yr = close_count / 252  # 252 trading days in a year
    # compounded annual growth rate
    CAGR = (close[-1] / close[0]) ** (1 / period_yr) - 1
    # drawdown, ulcer_index, max_drawdown = ulcer_index(close)
    CAGR_UI = CAGR / UI # original
    Std_UI = daily_return_std / UI  # larger is better
    CAGR_Std = CAGR / daily_return_std  # larger is better

    return period_yr, CAGR, CAGR_Std, CAGR_UI, daily_return_std, Std_UI, \
           drawdown, UI, max_drawdown

In [15]:
%%timeit 
for symbol in df_close.columns:
    close = df_close[symbol].values
    period_yr, CAGR, CAGR_Std, CAGR_UI, daily_return_std, Std_UI, \
    drawdown, UI, max_drawdown = myUI(close)
    # date_first = df_close.index[0].strftime('%Y-%m-%d')
    # date_last = df_close.index[-1].strftime('%Y-%m-%d')
    date_first = df_close.index[0]
    date_last = df_close.index[-1]
    cache = (symbol, date_first, date_last, period_yr, CAGR,
                ulcer_index, Std_UI, CAGR_Std, CAGR_UI)

664 µs ± 65.5 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
